<a href="https://colab.research.google.com/github/NjuChenZhiYu/grab_medical_firm_infos/blob/main/Huiqian_HW_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd


file_path = '/content/0808 Neuro Company-02.xlsx'

# 读取Excel文件
df = pd.read_excel(file_path)

# 读取Excel文件中的所有表格，返回一个字典
excel_dict = pd.read_excel(file_path, sheet_name=None)

# 查看所有表格的名称
print(excel_dict.keys())

# 读取目标表，比如表名为 '目标表名'
target_sheet_name = 'Neuro'
df_target = excel_dict[target_sheet_name]

# 打印目标表的前几行
print(df_target.head())


dict_keys(['Need PTA inputs>>', 'Neuro', '过程表1', '过程表2'])
   No Category                                       Company Name  Country  \
0   1    Neuro  MicroPort Magic Medical Technology (Shanghai) ...    China   
1   2    Neuro                                        Arthesys SA   France   
2   3    Neuro       Ningbo Guangyuan Zhixin Biotechnology Co Ltd    China   
3   4    Neuro                               Acandis GmbH & Co KG  Germany   
4   5    Neuro                                      Medtronic Plc  Ireland   

       Listed or not Listed or not.1  China Revenue Total revenue  \
0  Yes (HKEX: 00853)             Yes            NaN                 
1                 No              No            NaN           NaN   
2                 No              No            NaN           NaN   
3                 No              No            NaN           NaN   
4    Yes (NYSE: MDT)             Yes            NaN           NaN   

   国内是否和总代/operational solution provider 合作  总代名字  Source 

In [ ]:
# 找出重复的公司名称
duplicate_companies = df_target['Company Name'][df_target['Company Name'].duplicated(keep=False)]

# 获取唯一的重复公司名称列表
unique_duplicate_companies = duplicate_companies.unique()




In [ ]:
# !pip install fuzzywuzzy[speedup]


In [ ]:
# from fuzzywuzzy import process

# # 所有公司名
# company_names = list(unique_duplicate_companies)

# # 创建一个字典存放合并后的公司名（示例：用第一个出现的作为代表）
# merged_companies = {}

# # 遍历所有公司名，进行匹配和合并
# for name in company_names:
#     if name not in merged_companies:
#         # 找到与当前公司名字相似的公司
#         matches = process.extract(name, company_names, limit=None, score_cutoff=80)  # 90分以上视为相似
#         for match_name, score in matches:
#             if match_name not in merged_companies:
#                 merged_companies[match_name] = name  # 将相似的公司映射到同一公司名

# # 现在，替换原始公司名为映射的统一公司名
# df_target['Company Name'] = df_target['Company Name'].map(lambda x: merged_companies.get(x, x))

# # 查看合并后的公司名称
# print(df_target['Company Name'].unique())



In [ ]:
# 预定义的地区关键词（可以自己补充）
regions = ['shandong', 'nanjing', 'beijing', 'shanghai', 'guangdong', 'zhejiang', 'jiangsu','wuhan','shenzhen','suzhou','chengdu','chongqing'
           ,'xian','hemo','zhuhai','ningbo','hangzhou','shaanxi','guangzhou','tianjin','yantai','hunan','xi\'an','hebei','henan','sichuan','weihai'
           ,'liaoning','wuxi','zhuzhou','jinan','boston','changzhou']

# 创建一个映射字典，存放公司对应的地区
company_region_map = {}

company_list = df_target['Company Name'].tolist()

# 初始化一个字典，key是地区，value是公司列表
region_companies = {region: [] for region in regions}
region_companies['Unknown'] = []  # 针对未识别地区的公司

for name in company_list:
    region_found = 'Unknown'
    name_lower = name.lower()
    for region in regions:
        if 'changzhou' in name_lower :
            region_found = region
            break
        if region in name_lower :
            region_found = region
            break
    # 将公司添加到对应地区的列表中
    region_companies[region_found].append(name)

# 查看结果
for region, companies in region_companies.items():
    print(f"{region}: {companies}")

print(f"{'changzhou'}: {region_companies['changzhou']}")


shandong: ['Shandong Weigao Haixing Medical Equipment Co Ltd', 'Shandong Dazheng Medical Equipment Co Ltd', 'Changzhou Kefeng Medical Technology Co Ltd', 'Shandong Weixin Medical Device Co', 'Changzhou Kenite Medical Technology Co Ltd', 'Shandong Chuangwei Medical Technology Co Ltd', 'Shandong Huamei Technology Co Ltd', 'Changzhou Ruishenan Medical Equipment Co Ltd', 'Changzhou Kainet Medical Technology Co Ltd', 'Shandong Ande Medical Supplies Co Ltd', 'Shandong Pine Medical Equipment Co Ltd', 'Urai (Changzhou) Medical Technology Co Ltd', 'Shandong Weigao Medical Technology Co Ltd']
nanjing: ['Nanjing Maichuang Medical Technology Co Ltd', 'Nanjing Medico Technology Co Ltd', 'Pugao Medical Technology (Nanjing) Co Ltd', 'Nanjing Youdebang Medical Technology Co Ltd', 'Nanjing New Nuoint Medical Technology Co Ltd', 'Nanjing Newnoint Medical Technology Co Ltd', 'Puco Medical Technology (Nanjing) Co Ltd', 'Nanjing Smed Medical Technology Co Ltd', 'Nanjing SealMed Medical Technology Corp Ltd'

In [ ]:


# 假设你的公司列表（某地区的公司）如
company_lists = list(region_companies.values())

# 创建一个空字典存储公司映射关系
company_mapping = {}

# 简单的匹配规则：公司名是否包含相同的关键词（比如“Stryker”)
# 可以用集合的交集或者字符串包含判断

# 前提：先找到每个公司中的关键词（这里以公司名字中关键词为例，或者进行分词）
# 简单示例：用公司名字的第一个单词作为关键词

# 先提取所有公司名
all_companies = set()
for company_list in company_lists:
    all_companies.update(company_list)

# 初始化映射
for company in all_companies:
    company_mapping[company] = company  # 默认映射到自己

# 进行映射归一化
for company in all_companies:
    for other_company in all_companies:
        if company != other_company:
            # 简单规则：两个公司名包含相同关键词（此处示例以第一个词为关键词）
            if company.split()[0].lower() == other_company.split()[0].lower():
                # 统一映射到同一个“代表公司”
                # 这里我们可以选择用第一个公司作为代表
                company_mapping[other_company] = company_mapping[company]

# 最后输出映射关系
mapping_df = pd.DataFrame(list(company_mapping.items()), columns=['Company', 'Mapped Company'])
print(mapping_df)


                                     Company  \
0                     Micro Therapeutics Inc   
1          Achieva Medical (Shanghai) Co Ltd   
2                            Neuronetics Inc   
3                      Sequent Medical, Inc.   
4      Suzhou Hengrui Disheng Medical Co Ltd   
..                                       ...   
307                    MDCN Technologies Inc   
308     Hemu (China) Bioengineering Co., Ltd   
309                 Compumedics Germany GmbH   
310               Inomed Medizintechnik GmbH   
311  Shanghai Puhui Medical Equipment Co Ltd   

                                       Mapped Company  
0                              Micro Therapeutics Inc  
1                   Achieva Medical (Shanghai) Co Ltd  
2                                     Neuronetics Inc  
3                               Sequent Medical, Inc.  
4               Suzhou Hengrui Disheng Medical Co Ltd  
..                                                ...  
307                            

In [ ]:

!pip install openai


In [ ]:
# import openai

# # 替换为你的OpenAI API密钥
# openai.api_key = '你的OpenAI_API_Key'

# def ai_company_info(company_name):
#     prompt = f"请提供关于公司 {company_name} 的财务信息，包括China Revenue和Total Revenue。如果没有具体数据，请说明缺乏信息。"

#     response = openai.ChatCompletion.create(
#         model='gpt-4',  # 或 'gpt-3.5-turbo'
#         messages=[
#             {"role": "user", "content": prompt}
#         ],
#         temperature=0.3,
#         max_tokens=500
#     )
#     answer = response.choices[0].message['content']
#     return answer

# # 示例调用
# company_name = "Stryker corp"
# info = ai_company_info(company_name)
# print(info)


In [ ]:
pip install dashscope

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.4 MB/s eta 0:00:00


In [ ]:
from http import HTTPStatus
import dashscope

# 设置你的阿里云 API Key
dashscope.api_key = 'xxxx'  # 在阿里云控制台获取

# 调用 Qwen 模型
response = dashscope.Generation.call(
    model='qwen-max',
    prompt='你好，你是谁？'
)

if response.status_code == HTTPStatus.OK:
    print(response.output['text'])
else:
    print('请求失败:', response)

你好！我是Qwen，由阿里云开发的预训练语言模型。我被设计用来帮助人们生成各种类型的文本，如文章、故事、诗歌、故事等，并能够根据不同的场景和需求提供信息和支持。无论是创意写作还是寻求知识解答，我都在这里为你提供帮助。有什么我可以帮你的吗？


In [ ]:
# import os
# import openai

# openai.api_key = 'sk-16706c28ba9f42c2a13c6a28aea62e1e'

# client = OpenAI(
#     # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx",
#     api_key=os.getenv(api_key), # 如何获取API Key：https://help.aliyun.com/zh/model-studio/developer-reference/get-api-key
#     base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
# )


# completion = client.chat.completions.create(
#     # 模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
#     model="qwen-plus",  # qwen-plus 属于 qwen3 模型，如需开启思考模式，请参见：https://help.aliyun.com/zh/model-studio/deep-thinking
#     messages=[
#         {'role': 'system', 'content': 'You are a helpful assistant.'},
#         {'role': 'user', 'content': '你是谁？'}
#     ]
# )
# print(completion.choices[0].message.content)

In [ ]:
# import re
# import openai

# openai.api_key = '你的OpenAI_API_Key'


# # 全局参数
# MAX_TOTAL_COST = 6.0
# cost_per_1k_tokens = 0.002
# total_cost = 0.0

# import re

# def parse_answer(answer_text):
#     # 初始化返回值
#     same_relation = 'N'
#     related_company = ''
#     cooperation = 'N'
#     partner_name = ''

#     # 按行解析回答，寻找问答结构化的内容
#     for line in answer_text.splitlines():
#         # 问题1
#         q1_match = re.search(r'公司.*隶属关系\？\s+回答：([YN])', line)
#         if q1_match:
#             same_relation = q1_match.group(1).strip()
#             # 针对Y，尝试寻找具体公司名
#             if same_relation == 'Y':
#                 rel_match = re.search(r'隶属关系或为同一集团下的公司.*表明\s*([\w\s]+)', line)
#                 if rel_match:
#                     related_company = rel_match.group(1).strip()
#                 else:
#                     related_company = '未明确提及'

#         # 问题2
#         q2_match = re.search(r'公司.*合作\？\s+回答：([YN])', line)
#         if q2_match:
#             cooperation = q2_match.group(1).strip()
#             if cooperation == 'Y':
#                 coop_match = re.search(r'合作信息.*表明\s*([\w\s]+)', line)
#                 if coop_match:
#                     partner_name = coop_match.group(1).strip()
#                 else:
#                     partner_name = '未明确提及'

#     return same_relation, related_company, cooperation, partner_name




# def get_company_info(company_name):
#     global total_cost
#     prompt = f"""请提供关于公司 {company_name} 的财务信息，包括 "China Revenue" 和 "Total Revenue"。
# 同时请回答以下问题：
# 1. 该公司是否与之前出现过的公司属于同一公司或有隶属关系？请回答 'Y' 或 'N'，若有，请提供关联公司的名称。
# 2. 该公司是否与国内的总代/经销商/平台服务商合作？如果有，请回答 'Y'，否则 'N'。若有合作，请提供总代公司名称。
# 3. 该公司数据来源
# """

#     estimated_cost = (600 / 1000) * cost_per_1k_tokens
#     if total_cost + estimated_cost > MAX_TOTAL_COST:
#         print("已达到预算上限，停止请求。")
#         return None, None, 'N', '', '', ''
#     response = dashscope.Generation.call(
#         model='qwen-plus',
#         input={
#             'messages': [
#                 {'role': 'user', 'content': prompt}
#             ]
#         },
#         parameters={
#             'max_tokens': 150,
#             'temperature': 0.7
#         }
#     )
#     if response.status_code == HTTPStatus.OK:
#         print(response.output['text'])
#     else:
#         print('请求失败:', response)
#     answer = response.output['text']
#     total_cost += estimated_cost

#     # 用户自定义解析函数
#     same_relation, related_company, cooperation, partner_name = parse_answer(answer)

#     # 提取财务信息
#     china_rev, total_rev = '未获取', '未获取'
#     for line in answer.splitlines():
#         if 'China Revenue' in line:
#             china_rev = line
#         elif 'Total Revenue' in line:
#             total_rev = line

#     return china_rev, total_rev, same_relation, related_company, cooperation, partner_name, answer

# # 使用示例（遍历公司列表）
# # 请确保在此之前定义了你的公司列表
# results = []

# for idx,company in enumerate(company_list[28: ]):
#     if total_cost >= MAX_TOTAL_COST:
#         print("查询到第 ：" + str(idx + 1) + "个数据")
#         print("费用用尽，停止查询。")
#         break
#     print("查询到第 ：" + str(idx + 1) + "个数据")
#     print("公司名称： " + company)

#     china_rev, total_rev, same_relation, related_company, cooperation, partner_name, full_answer = get_company_info(company)
#     results.append({
#         '公司': company,
#         '中国收入': china_rev,
#         '总收入': total_rev,
#         '是否存在之前出现的关系': same_relation,
#         '关联公司': related_company,
#         '合作': cooperation,
#         '总代名字': partner_name,
#         'API回答': full_answer,
#         'source': 'AI 模型'
#     })

# # 保存到CSV
# import pandas as pd
# df_results = pd.DataFrame(results)
# df_results.to_csv('公司信息补充.csv', index=False)
# print("处理完成，已保存到 '公司信息补充.csv'。")


In [ ]:
company_lists = list(df_target['Company Name'])
len(company_lists)

312

In [ ]:
import re
import openai

openai.api_key = '你的OpenAI_API_Key'


# 全局参数
MAX_TOTAL_COST = 6.0
cost_per_1k_tokens = 0.002
total_cost = 0.0

import re



def get_company_info(company_name):
    global total_cost
    prompt = f"""请提供关于公司 {company_name} 的财务信息，包括 'China Revenue' 和 'Total Revenue',NA 代表没有
    同时请回答以下问题：
    1. 该公司数据来源, 请回答 'source' : xxxx
    2.
    回答不超过150字
    """

    estimated_cost = (600 / 1000) * cost_per_1k_tokens
    if total_cost + estimated_cost > MAX_TOTAL_COST:
        print("已达到预算上限，停止请求。")
        return None, None, 'N', '', '', ''
    response = dashscope.Generation.call(
        model='qwen-max',
        messages=[
            {'role': 'user', 'content': prompt}
        ],
        parameters={
            'max_tokens': 120,
            'temperature': 0.7
        }
    )
    if response.status_code == HTTPStatus.OK:
        print(response.output['text'])
    else:
        print('请求失败:', response)
    answer = response.output['text']
    total_cost += estimated_cost

    # 用户自定义解析函数
    #same_relation, related_company, cooperation, partner_name = parse_answer(answer)

    # 提取财务信息
    china_rev, total_rev = '未获取', '未获取'
    for line in answer.splitlines():
        if 'China Revenue' in line:
            china_rev = line
        elif 'Total Revenue' in line:
            total_rev = line

    return china_rev, total_rev, answer

# 使用示例（遍历公司列表）
# 请确保在此之前定义了你的公司列表
results = []

for idx,company in enumerate(company_lists):
    if total_cost >= MAX_TOTAL_COST:
        print("查询到第 ：" + str(idx + 1) + "个数据")
        print("费用用尽，停止查询。")
        break
    print("查询到第 ：" + str(idx + 1) + "个数据")
    print("公司名称： " + company)

    china_rev, total_rev, full_answer = get_company_info(company)
    results.append({
        'number': idx + 1,
        '公司': company,
        '中国收入': china_rev,
        'API回答': full_answer,
    })

# 保存到CSV
import pandas as pd
df_results = pd.DataFrame(results)
df_results.to_csv('公司信息补充.csv', index=False)
print("处理完成，已保存到 '公司信息补充.csv'。")


查询到第 ：1个数据
公司名称： MicroPort Magic Medical Technology (Shanghai) Co Ltd
关于公司 MicroPort Magic Medical Technology (Shanghai) Co Ltd 的财务信息，公开资料中没有直接提供具体的 'China Revenue' 和 'Total Revenue' 数据。因此，对于您的请求，我只能给出如下回答：'China Revenue': NA, 'Total Revenue': NA。

1. 该公司数据来源: 由于具体财务数字未在公开渠道明确列出，建议直接访问其官方网站或查阅最新的官方财务报告以获取最准确的信息。'source': 官方网站/官方财务报告。
查询到第 ：2个数据
公司名称： Arthesys SA
关于公司Arthesys SA的具体财务信息，包括'China Revenue'和'Total Revenue'的数据，在公开渠道上并不容易获取到。因此，对于这两个数值，我只能给出如下回答：'China Revenue': NA, 'Total Revenue': NA。至于该公司数据的来源，由于缺乏具体信息支持，同样无法提供准确答案，故'source': NA。建议直接访问Arthesys SA的官方网站或联系其官方客服以获得更准确的信息。
查询到第 ：3个数据
公司名称： Ningbo Guangyuan Zhixin Biotechnology Co Ltd
关于公司Ningbo Guangyuan Zhixin Biotechnology Co Ltd的具体财务信息，包括'China Revenue'和'Total Revenue'，目前我没有直接访问到最新的具体数值。因此，根据现有信息：'China Revenue' : NA, 'Total Revenue' : NA。对于此类特定公司的详细财务数据，建议查阅官方财报或相关金融信息服务平台获取最准确的信息。'source' : 无直接公开来源提供具体数字。
查询到第 ：4个数据
公司名称： Acandis GmbH & Co KG
关于Acandis GmbH & Co KG的具体财务信息，包括中国地区的收入('China Revenue')和总收入('Total Revenue')，公开

ERROR:dashscope:


KeyboardInterrupt: 

In [ ]:
df_results = pd.DataFrame(results)
df_results.to_csv('公司信息补充.csv', index=False)
print("处理完成，已保存到 '公司信息补充.csv'。")

处理完成，已保存到 '公司信息补充.csv'。


In [ ]:
import pandas as pd
import re

import pandas as pd
import re

def parse_api_response(response):
    # 提取总收入
    total_revenue_match = re.search(r'总[营收入]?[：:]?\s*(约)?\s*([\d.]+)亿[元美]?元?', response)
    total_revenue = float(total_revenue_match.group(2)) if total_revenue_match else 'NA'

    # 提取中国区收入
    china_revenue_match = re.search(r'中国[区市场]?收入[：:]?\s*(约)?\s*([\d.]+)亿[元美]?元?', response)
    china_revenue = float(china_revenue_match.group(2)) if china_revenue_match else 'NA'

    # 判断是否有合作伙伴
    has_partner = 'Y' if re.search(r'1\.\s*Y', response, re.IGNORECASE) else 'N'

    # 提取合作伙伴名称
    partner_name = re.search(r'1\.\s*Y\s*[（(](.+?)[）)]', response)
    if not partner_name:
        partner_name = re.search(r'合作[方总代]?[包括：:]\s*(.+?)(?:。|\n|$)', response)
    partner_name = partner_name.group(1).strip() if partner_name else None

    # 提取数据来源
    source = re.search(r'(数据)?来源[：:]\s*(.+?)(?:。|\n|$)', response)
    source = source.group(2).strip() if source else '未提供'

    return {
        'china_revenue': china_revenue,
        'total_revenue': total_revenue,
        'has_partner': has_partner,
        'partner_name': partner_name,
        'source': source
    }

# 读取CSV文件
qwen_df = pd.read_csv('/content/公司信息补充.csv')

# 应用解析函数到'API回答'列
parsed_data = qwen_df['API回答'].apply(parse_api_response)

# 将解析后的数据添加到原始DataFrame
for key in ['china_revenue', 'total_revenue', 'has_partner', 'partner_name', 'source']:
    qwen_df[key] = parsed_data.apply(lambda x: x[key])

# 显示结果
#print(qwen_df[['API回答', 'china_revenue', 'total_revenue', 'has_partner', 'partner_name', 'source']])


# 读取你的csv（假设路径为 data.csv）
qwen_df = pd.read_csv('/content/公司信息补充.csv')

company_answer_dict = dict(zip(qwen_df['公司'], qwen_df['API回答']))

for idx, row in df_target.iterrows():
    company_name = row['Company Name']
    answer_text = company_answer_dict.get(company_name, '')  # 获取对应的API回答内容
    df_target.loc[idx,'Comment'] = answer_text
    #attrs = parse_api_response(answer_text)
    # print(answer_text)
    # for key, value in attrs.items():
    #     print(f"{key}: {value}")
    # df_target.loc[idx, 'China Revenue'] = attrs['china revenue']
    # df_target.loc[idx, 'Total revenue'] = attrs['total revenue']
    # df_target.loc[idx, '国内是否和总代/operational solution provider合作'] = attrs['has_partner']
    # df_target.loc[idx, '总代名字'] = attrs['partner_name']
    # df_target.loc[idx, 'source'] = attrs['source']
    # print()


# 保存结果
df_target.to_csv('更新后数据.csv', index=False)
